In [1]:
import liana as li
import scanpy as sc
from anndata import AnnData
import pandas as pd
from liana.mt import rank_aggregate
from liana.method import singlecellsignalr, connectome, cellphonedb, natmi, logfc, cellchat, geometric_mean
from pycirclize import Circos

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# loading original anndata
pvat_8weeks_hf_male = sc.read_h5ad('../../exported_data/pvat_8weeks_hf_male.h5ad')
pvat_8weeks_hf_male

AnnData object with n_obs × n_vars = 9608 × 20743
    obs: 'sample_id', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', 'n_genes', 'scDblFinder_score', 'scDblFinder_class', 'Sample', 'tissue', 'diet', 'time', 'sex', 'batch', 'sample_type', 'rat_id', 'weight_3W', 'weight_10W', 'weight_17W', 'weight_24W', 'MAP_7W', 'systolic_bp_7W', 'MAP_9W', 'systolic_bp_9W', 'MAP_16W', 'systolic_bp_16W', 'MAP_24W', 'systolic_bp_24W', 'doublet', 'celltype', 'celltype_broad'
    var: 'gene_ids', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells'
    uns: 'cellchat_interaction_table_count', 'cellchat_res', 'cellchat_res_filtered', 

In [3]:
pvat_8weeks_hf_male.obs['celltype_broad'].unique().tolist()

['Fibroblasts',
 'Adipocytes',
 'Endothelial_Cells',
 'Immune_Cells',
 'Mesothelial_Cells',
 'Pericytes',
 'SMCs',
 'Neuronal_Cells']

In [4]:
# List available methods in lliana framework
li.mt.show_methods()

,Method Name,Magnitude Score,Specificity Score,Reference
0,CellPhoneDB,lr_means,cellphone_pvals,"Efremova, M., Vento-Tormo, M., Teichmann, S.A...."
0,Connectome,expr_prod,scaled_weight,"Raredon, M.S.B., Yang, J., Garritano, J., Wang..."
0,log2FC,None,lr_logfc,"Dimitrov, D., Türei, D., Garrido-Rodriguez, M...."
0,NATMI,expr_prod,spec_weight,"Hou, R., Denisenko, E., Ong, H.T., Ramilowski,..."
0,SingleCellSignalR,lrscore,None,"Cabello-Aguilar, S., Alame, M., Kon-Sun-Tack, ..."
0,CellChat,lr_probs,cellchat_pvals,"Jin, S., Guerrero-Juarez, C.F., Zhang, L., Cha..."
0,Rank_Aggregate,magnitude_rank,specificity_rank,"Dimitrov, D., Türei, D., Garrido-Rodriguez, M...."
0,Geometric Mean,lr_gmeans,gmean_pvals,CellPhoneDBv2's permutation approach applied t...


In [5]:
pvat_8weeks_hf_male.raw = pvat_8weeks_hf_male
pvat_8weeks_hf_male.raw.X

<9608x20743 sparse matrix of type '<class 'numpy.float64'>'
	with 12908119 stored elements in Compressed Sparse Row format>

In [6]:
# Run cellchat analysis on PVAT data
cellchat(pvat_8weeks_hf_male, groupby='celltype_broad', expr_prop=0.1, resource_name='mouseconsensus', verbose=True, key_added='cellchat_res')

Using `.raw`!
/Users/pandavis/miniconda3/lib/python3.11/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
972 features of mat are empty, they will be removed.
Make sure that normalized counts are passed!
/Users/pandavis/miniconda3/lib/python3.11/site-packages/liana/method/_pipe_utils/_pre.py:148: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
['Cyp2c6_v1-ps2'] contain `_`. Consider replacing those!
Using resource `mouseconsensus`.
0.18 of entities in the resource are missing from the data.


Generating ligand-receptor stats for 9608 samples and 19771 features


100%|████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:34<00:00,  6.46it/s]


In [7]:
ligand_receptor_pvat = pvat_8weeks_hf_male.uns['cellchat_res']
ligand_receptor_pvat.head()

,ligand,ligand_complex,ligand_props,ligand_trimean,mat_max,receptor,receptor_complex,receptor_props,receptor_trimean,source,target,lr_probs,cellchat_pvals
69,Vegfa,Vegfa,0.955981,0.021484,256.0,Nrp1,Nrp1,0.782081,0.009766,Adipocytes,Adipocytes,0.000419,0.0
7264,Sorbs1,Sorbs1,0.933893,0.025391,256.0,Itga1,Itga1,0.875000,0.007812,Adipocytes,SMCs,0.000397,0.0
7713,Lama2,Lama2,0.918292,0.021484,256.0,Itga1,Itga1,0.875000,0.007812,Fibroblasts,SMCs,0.000336,0.0
617,C4a,C4a,0.953368,0.014648,256.0,Nrp1,Nrp1,0.782081,0.009766,Mesothelial_Cells,Adipocytes,0.000286,0.0
38,Ntrk3,Ntrk3,0.933271,0.026367,256.0,Ptprs,Ptprs,0.609115,0.003906,Adipocytes,Adipocytes,0.000206,0.0


In [8]:
ligand_receptor_string = (ligand_receptor_pvat.loc[:,'ligand_complex'].str.contains('Col')) & (ligand_receptor_pvat.loc[:,'receptor_complex'].str.contains('Itg'))
cell_source_string = ligand_receptor_pvat.loc[:,'source'] == 'Fibroblasts'
cell_target_string = ligand_receptor_pvat.loc[:,'target'] == 'Adipocytes'
cell_pvals = ligand_receptor_pvat.loc[:,'cellchat_pvals'] <= 0.05

ligand_receptor_pvat[ligand_receptor_string & cell_source_string & cell_target_string & cell_pvals]

,ligand,ligand_complex,ligand_props,ligand_trimean,mat_max,receptor,receptor_complex,receptor_props,receptor_trimean,source,target,lr_probs,cellchat_pvals
309,Col3a1,Col3a1,0.828227,0.011719,256.0,Itgb1,Itga1_Itgb1,0.264116,0.000977,Fibroblasts,Adipocytes,0.000023,0.0
303,Col5a3,Col5a3,0.771588,0.011719,256.0,Itgb1,Itga1_Itgb1,0.264116,0.000977,Fibroblasts,Adipocytes,0.000023,0.0
273,Col4a1,Col4a1,0.643454,0.006836,256.0,Itgb1,Itga1_Itgb1,0.264116,0.000977,Fibroblasts,Adipocytes,0.000013,0.0
285,Col6a3,Col6a3,0.656453,0.005859,256.0,Itgb1,Itga1_Itgb1,0.264116,0.000977,Fibroblasts,Adipocytes,0.000011,0.0
283,Col6a2,Col6a2,0.691736,0.004883,256.0,Itgb1,Itga1_Itgb1,0.264116,0.000977,Fibroblasts,Adipocytes,0.000010,0.0
267,Col6a6,Col6a6,0.525534,0.004883,256.0,Itgb1,Itga1_Itgb1,0.264116,0.000977,Fibroblasts,Adipocytes,0.000010,0.0
305,Col5a2,Col5a2,0.709378,0.004883,256.0,Itgb1,Itga1_Itgb1,0.264116,0.000977,Fibroblasts,Adipocytes,0.000010,0.0
307,Col5a1,Col5a1,0.644383,0.003906,256.0,Itgb1,Itga1_Itgb1,0.264116,0.000977,Fibroblasts,Adipocytes,0.000008,0.0
281,Col6a1,Col6a1,0.620241,0.003906,256.0,Itgb1,Itga1_Itgb1,0.264116,0.000977,Fibroblasts,Adipocytes,0.000008,0.0
277,Col1a2,Col1a2,0.600743,0.003906,256.0,Itgb1,Itga1_Itgb1,0.264116,0.000977,Fibroblasts,Adipocytes,0.000008,0.0


In [9]:
ligand_receptor_pvat.to_csv("pvat_8weeks_hf_male_cellchat.csv")
pvat_8weeks_hf_male.write("../../exported_data/pvat_8weeks_hf_male.h5ad")